In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as kr
import collections
import re
import jieba
import math
import os
import random
import logging
import time
tf.logging.set_verbosity(tf.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
logger = logging.getLogger('bert-log')
logger.setLevel(logging.DEBUG)
timestamp = time.strftime("%Y.%m.%d_%H.%M.%S", time.localtime())
fh = logging.FileHandler('bert.txt')
fh.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s][%(levelname)s] ## %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
logger.addHandler(fh)
logger.addHandler(ch)

In [3]:
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [4]:

from importlib import reload
import config
reload(config)
cnn_config = config.CNNConfig

In [5]:
# def feed_data(x_batch, y_batch, keep_prob):
#     feed_dict = {
#         model.input_x: x_batch,
#         model.input_y: y_batch,
#         model.keep_prob: keep_prob
#     }
#     return feed_dict

# def feed_data(x_batch, y_batch, keep_prob):
#     feed_dict = {
#         model.input_x: x_batch,
#         model.input_y: y_batch,
#         model.keep_prob: keep_prob
#     }
#     return feed_dict

def read_file(file_dir, train=True):
    """
    读取csv文件
    """
    if train:
        comments, labels = [], []
        df_train = pd.read_csv(file_dir, sep='\t')
        for comment, label in zip(df_train.comment, df_train.label):
            comments.append(comment)
            labels.append(label)
        return comments, labels
    else:
        comments, comment_ids = [], []
        df_test = pd.read_csv(file_dir)
        for comment, comment_id in zip(df_test.comment, df_test.id):
            comments.append(comment)
            comment_ids.append(comment_id)
        return comments, comment_ids
    
def build_vocab(train_comments, vocab_size=5000):
    """
    对训练集分词,统计词频并取vocab_size个词作为词表
    """
    
    all_words = []
    
    for comment in train_comments:
        seg_comment = jieba.lcut(comment)
        all_words.extend(seg_comment)
    word_counter = collections.Counter(all_words)
    select_words = word_counter.most_common(vocab_size-2)
    select_words, _ = list(zip(*select_words))
    select_words = ["<PAD>"] + ["<UNK>"] + list(select_words)
    word2id = {word: idx for idx, word in enumerate(select_words)}
    
    return word2id
    
def convert_to_inputids(comments, word2id, max_len=40):
    
    input_ids = []
    
    for comment in comments:
        comment_ids = [word2id[w] if w in word2id  else word2id["<UNK>"] for w in comment]
        input_ids.append(comment_ids)
        
    input_ids = kr.preprocessing.sequence.pad_sequences(input_ids, max_len, padding='post')
    
    return input_ids

def batch_iter(x, y, batch_size=32, shuffle=False):
    """
    batch数据生成器
    """

    sample_len = len(x)
    batch_count = math.ceil(sample_len / batch_size)
    # 随机打散
    if shuffle:
        indices = np.random.permutation(np.arange(sample_len))
    else:
        indices = list(np.arange(sample_len))
    x_shuffle = np.array(x)[indices]
    y_shuffle = np.array(y)[indices]

    while True:
        for i in range(batch_count):
            start_id = i * batch_size
            end_id = min((i + 1) * batch_size, sample_len)
            yield x_shuffle[start_id: end_id], y_shuffle[start_id: end_id]  
            
def test_batch_iter(x, batch_size=32):
    
    sample_len = len(x)
    batch_count = math.ceil(sample_len / batch_size)
    
    x = np.array(x)
    while True:    
        for i in range(batch_count):
            start_id = i * batch_size
            end_id = min((i + 1) * batch_size, sample_len)
            yield x[start_id: end_id]

train_comments, labels = read_file(cnn_config.train_dir)
word2id = build_vocab(train_comments)
train_ids = convert_to_inputids(train_comments, word2id, 50)
data_iter = batch_iter(train_ids, labels)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.647 seconds.
Prefix dict has been built successfully.


In [6]:
# test_comments, test_comment_ids = read_file(test_path, train=False)
# test_ids = convert_to_inputids(test_comments, word2id, 50)
# test_loader = test_batch_iter(test_ids, rnn_config.test_batch_size)

In [7]:
test_x, test_y = next(data_iter)

In [8]:
class BiRNN(object):
    
    def __init__(self, config, init_emd=None, is_training=True, is_testing=False):
        
        self.config = config
        self.init_emd = init_emd
        self.is_training = is_training
        self.is_testing = is_testing
#         self.learning_rate = self.config.learning_rate
        self.learning_rate = tf.Variable(self.config.learning_rate, trainable=False)
        
        self.input_x = tf.placeholder(tf.int32, [None, self.config.seq_length], name='input_x')
        self.input_y = tf.placeholder(tf.int32, name='input_y')
        self.keep_prob = tf.placeholder(tf.int32, name='keep_prob')
        
        self.forward()
        
    def forward(self):
        
        def lstm_cell():
            return tf.nn.rnn_cell.LSTMCell(self.config.hidden_dim)
        
        def gru_cell():
            return tf.nn.rnn_cell.GRUCell(self.config.hidden_dim)
        
        
        with tf.device('/cpu:0'):
            if self.init_emd is not None:
                embedding = tf.Variable(init_emd, trainable=True, name="emb", dtype=tf.float32)
            else:
                init_mat = tf.random_uniform([self.config.vocab_size, self.config.hidden_dim], -1, 1)
                embedding = tf.Variable(init_mat, trainable=True, name="emb", dtype=tf.float32)
            
            # [batch_size, seq_len, emb_dim]
            inputs_emb = tf.nn.embedding_lookup(embedding, self.input_x)
            
        with tf.name_scope("birnn_cell"):
            if self.config.rnn.lower() == 'lstm':
                cell_fw = lstm_cell()
                cell_bw = lstm_cell()
            else:
                cell_fw = gru_cell()
                cell_bw = gru_cell()

            if self.is_training:
                cell_fw = tf.nn.rnn_cell.DropoutWrapper(cell_fw, 
                                            output_keep_prob=self.config.dropout_keep_prob)
                cell_bw = tf.nn.rnn_cell.DropoutWrapper(cell_bw, 
                                            output_keep_prob=self.config.dropout_keep_prob) 
                
            cell_fw = tf.nn.rnn_cell.MultiRNNCell([cell_fw] * self.config.num_layers)
            cell_bw = tf.nn.rnn_cell.MultiRNNCell([cell_bw] * self.config.num_layers)
            
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                          cell_bw, 
                                                                          inputs_emb, 
                                                                          dtype=tf.float32,
                                                                          scope="bi-lstm")
            outputs = tf.concat(outputs, -1)
            outputs = tf.reshape(outputs, 
                                 [-1, self.config.seq_length, self.config.hidden_dim * 2])
            
            
        with tf.name_scope("pooling"):
            # [batch_size, 1, hid_size * 2]
            outputs_max = tf.nn.max_pool1d(outputs,
                                  ksize=self.config.seq_length, 
                                  strides=1, 
                                  padding="VALID")
            
            # [batch_size, 1, hid_size * 2]
            outputs_avg = tf.nn.avg_pool1d(outputs,
                                  ksize=self.config.seq_length, 
                                  strides=1, 
                                  padding="VALID")

#             avg_pool_res = tf.reduce_max(outputs,  reduction_indices=[1])
#             max_pool_res = tf.reduce_mean(outputs. reduction_indices=[1])
            
            # [batch_size, hid_size * 2]
            outputs_max = tf.squeeze(outputs_max, 1)
            outputs_avg = tf.squeeze(outputs_avg, 1)
            # [batch_size, hid_size * 4]
            pooling_outputs = tf.concat([outputs_max, outputs_avg], 1)
            
        with tf.name_scope("logits"):
            # [batch_size, num_classes]
            self.logits = tf.layers.dense(pooling_outputs, self.config.num_classes)
            self.logits_outputs = tf.nn.softmax(self.logits, axis=1)
            self.y_pred = tf.argmax(self.logits, axis=1)
        
        if self.is_testing: return 
        
        with tf.name_scope("loss"):
            self.labels_onehot = tf.one_hot(self.input_y, depth=self.config.num_classes)
            
#             cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.input_y)
            cross_entropy = tf.losses.softmax_cross_entropy(onehot_labels=self.labels_onehot, 
                                              logits=self.logits) # label_smoothing=0.001
            self.loss = tf.reduce_mean(cross_entropy)
               
        with tf.name_scope("train_op"):
            self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
            self.train_op = self.optimizer.minimize(self.loss)
            
        with tf.name_scope("acc"):
            _, self.cur_precision = tf.metrics.precision(self.input_y, self.y_pred)
            
        with tf.name_scope("recall"):
            _, self.cur_recall = tf.metrics.recall(self.input_y, self.y_pred)
            
        with tf.name_scope("f1"):
            self.cur_f1 = 2 * self.cur_precision * self.cur_recall / (self.cur_precision + self.cur_recall)
            
        with tf.name_scope("new_lr"):
            # 用于更新 学习率
            self.new_lr = tf.placeholder(tf.float32, shape=[])
            # 将new_lr赋值给lr_update
            self.lr_update = tf.assign(self.learning_rate, self.new_lr)            
            
    # 更新 学习率
    def assign_lr(self, session, lr_value):
        session.run(self.lr_update, feed_dict={self.new_lr: lr_value})
                
import config
reload(config)
config = config.RNNConfig

# tf.reset_default_graph()
# with tf.Session() as sess:
#     model = BiRNN(config)
#     feed_dict = {
#         model.input_x: test_x,
#         model.input_y: test_y,
#         model.keep_prob: 0.2
#         }
#     sess.run(tf.local_variables_initializer()) 
#     sess.run(tf.global_variables_initializer())
#     outputs = sess.run(model.test, feed_dict=feed_dict)
# outputs       

In [10]:
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from tqdm import tqdm
from sklearn.metrics import f1_score, recall_score, precision_score


def evaluate(sess, eval_loader, eval_allsteps):
    total_loss = 0.0
    total_f1 = 0.0
    for step, (x_batch, y_batch) in tqdm(enumerate(eval_loader)):
        if step > eval_allsteps:
            break
        feed_dict = {
            model.input_x: x_batch,
            model.input_y: y_batch,
            model.keep_prob: 1.0
            }
        cur_loss, cur_f1 = sess.run([model.loss, model.cur_f1], 
                                    feed_dict=feed_dict)
        total_loss += total_loss
        
    return cur_f1, cur_loss
            
    

def train_model(config, model, all_ids, all_labels):
    tensorboard_dir = config.tensorboard_dir
    
    # 配置tensorboard
    tf.summary.scalar("loss", model.loss)
    tf.summary.scalar("cur_precision", model.cur_precision)
    tf.summary.scalar("cur_recall", model.cur_recall)
    tf.summary.scalar("cur_f1", model.cur_f1)
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(tensorboard_dir)
    
    # 配置saver
    saver = tf.train.Saver()
    orig_decay = config.lr_decay
    learning_rate = config.learning_rate
    max_lr_epoch = config.max_lr_epoch
    
    # 构建训练集，验证集
    all_labels = np.array(all_labels)
    kf = StratifiedKFold(
        n_splits=config.fold_count, random_state=config.seed, shuffle=True).split(X=all_ids, y=all_labels)
    
    # 划分训练验证集
    for fold_idx, (train_idx, eval_idx) in enumerate(kf):
        # 创建session
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
#         sess.run(tf.local_variables_initializer())
        writer.add_graph(sess.graph)
        
        # 当前折的训练集
        train_ids = all_ids[train_idx]
        train_labels = all_labels[train_idx]
        train_loader = batch_iter(train_ids, train_labels)
        # 当前折的验证集
        eval_ids = all_ids[eval_idx]
        eval_labels = all_labels[eval_idx]
        eval_loader = batch_iter(eval_ids, eval_labels, batch_size=64)
        
        best_f1 = -999
        earlystop_count = 0
        train_steps_fold = math.ceil(len(train_ids) / config.train_batch_size)
        eval_steps_fold = math.ceil(len(eval_ids) / config.eval_batch_size)
        for epoch in range(config.num_epochs):
            logger.info("epoch:{}, fold_idx:{}".format(epoch, fold_idx))
            # 初始化matrics算子
            sess.run(tf.local_variables_initializer())
            new_lr_decay = orig_decay ** max(epoch + 1 - max_lr_epoch, 0)
            model.assign_lr(sess, learning_rate * new_lr_decay)
            logger.info(sess.run(model.learning_rate))
            if earlystop_count >= config.early_stop:
                break
                
            for step, (x_batch, y_batch) in tqdm(enumerate(train_loader)):
                feed_dict = {
                    model.input_x: x_batch,
                    model.input_y: y_batch,
                    model.keep_prob: config.dropout_keep_prob
                    }
                if step % config.save_per_batch == 0  and epoch == 0 and fold_idx == 0:
                    # 每多少轮次将训练结果写入tensorboard scalar
                    s = sess.run(merged_summary, feed_dict=feed_dict)
                    writer.add_summary(s, step)
                    
                # 反向传播迭代优化
                feed_dict[model.keep_prob] = config.dropout_keep_prob
                _, cur_f1 = sess.run([model.train_op, model.cur_f1], feed_dict=feed_dict)
                
                # 迭代step次时，实际已经完整的预测了traindata
                # 如sample/bs = 62.5需要迭代63次，step=62时已经迭代63次                
                if step + 1 >= train_steps_fold:
                    break
            logger.info("train_f1:{}".format(cur_f1))
            # 初始化matrics算子，进行eval
            sess.run(tf.local_variables_initializer())
            # 对验证集进行预测评估，并保存模型
            eval_f1, eval_loss = evaluate(sess, eval_loader, eval_steps_fold)
            logger.info("epoch:{}, eval_f1:{}".format(epoch, eval_f1))
            
            if cur_f1 > best_f1:
                best_f1 = cur_f1
                earlystop_count = 0
                saver.save(sess=sess, save_path=config.save_dir + config.sava_model_name + '{}'.format(fold_idx))
            else:
                earlystop_count += 1

                
            
tf.reset_default_graph()
model = BiRNN(config)    
train_comments, train_labels = read_file(config.train_dir)
word2id = build_vocab(train_comments)
train_ids = convert_to_inputids(train_comments, word2id, cnn_config.seq_length)

train_model(config, model, train_ids, train_labels)

[2021-03-28 19:05:50,938][INFO] ## epoch:0, fold_idx:0
[2021-03-28 19:05:51,046][INFO] ## 0.01
273it [00:50,  5.37it/s]
[2021-03-28 19:06:41,848][INFO] ## train_f1:0.7142361998558044
41it [00:03, 10.63it/s]
[2021-03-28 19:06:45,774][INFO] ## epoch:0, eval_f1:0.854521632194519
[2021-03-28 19:06:45,915][INFO] ## epoch:1, fold_idx:0
[2021-03-28 19:06:45,979][INFO] ## 0.01
273it [00:45,  5.99it/s]
[2021-03-28 19:07:31,571][INFO] ## train_f1:0.8598276972770691
41it [00:02, 13.95it/s]
[2021-03-28 19:07:34,587][INFO] ## epoch:1, eval_f1:0.8362184166908264
[2021-03-28 19:07:34,695][INFO] ## epoch:2, fold_idx:0
[2021-03-28 19:07:34,766][INFO] ## 0.01
273it [00:47,  5.75it/s]
[2021-03-28 19:08:22,223][INFO] ## train_f1:0.8931739330291748
41it [00:02, 14.03it/s]
[2021-03-28 19:08:25,223][INFO] ## epoch:2, eval_f1:0.8475034236907959
[2021-03-28 19:08:25,343][INFO] ## epoch:3, fold_idx:0
[2021-03-28 19:08:25,410][INFO] ## 0.01
273it [00:44,  6.17it/s]
[2021-03-28 19:09:09,642][INFO] ## train_f1:0.9

41it [00:02, 15.06it/s]
[2021-03-28 19:30:25,760][INFO] ## epoch:5, eval_f1:0.8085106015205383
[2021-03-28 19:30:26,066][INFO] ## epoch:0, fold_idx:5
[2021-03-28 19:30:26,181][INFO] ## 0.01
273it [00:46,  5.87it/s]
[2021-03-28 19:31:12,660][INFO] ## train_f1:0.6934801340103149
41it [00:03, 12.98it/s]
[2021-03-28 19:31:15,894][INFO] ## epoch:0, eval_f1:0.8108108043670654
[2021-03-28 19:31:16,038][INFO] ## epoch:1, fold_idx:5
[2021-03-28 19:31:16,105][INFO] ## 0.01
273it [00:45,  6.02it/s]
[2021-03-28 19:32:01,490][INFO] ## train_f1:0.8486528992652893
41it [00:02, 14.10it/s]
[2021-03-28 19:32:04,473][INFO] ## epoch:1, eval_f1:0.8355554938316345
[2021-03-28 19:32:04,580][INFO] ## epoch:2, fold_idx:5
[2021-03-28 19:32:04,650][INFO] ## 0.01
273it [00:47,  5.71it/s]
[2021-03-28 19:32:52,481][INFO] ## train_f1:0.8958009481430054
41it [00:02, 14.88it/s]
[2021-03-28 19:32:55,314][INFO] ## epoch:2, eval_f1:0.8504801392555237
[2021-03-28 19:32:55,430][INFO] ## epoch:3, fold_idx:5
[2021-03-28 19:3

In [ ]:
# test_comments, test_comment_ids = read_file(test_path, train=False)
# test_ids = convert_to_inputids(test_comments, word2id, 50)
# test_loader = test_batch_iter(test_ids, rnn_config.test_batch_size)
# tf.metrics.precision.reset()

In [16]:
def test_model(config, model, test_ids):
    
    test_loader = test_batch_iter(test_ids, config.test_batch_size)
    test_steps_fold = math.ceil(len(test_ids) / config.test_batch_size)
    df_result = pd.read_csv(config.sample_dir)
    
    assert len(test_ids) == len(df_result), "check your pred_data!"
    test_logits = np.zeros((len(df_result), 2))
    
    for fold_idx in range(config.fold_count):
        model_path = config.save_dir + config.sava_model_name + '{}'.format(fold_idx)
        print(model_path)
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        saver.restore(sess=sess, save_path=model_path)

        for step, x_batch in tqdm(enumerate(test_loader)): 
                
            start_id = step * config.test_batch_size
            end_id = min((step + 1) * config.test_batch_size, len(df_result))
            
            feed_dict = {
                model.input_x: x_batch,
                model.input_y: 0.0,
                model.keep_prob: 1.0
                }
            logits_batch, pred_batch = sess.run([model.logits_outputs, model.y_pred],
                                                feed_dict=feed_dict)
#             test_preds.extend(pred_batch)
            test_logits[start_id: end_id] += logits_batch
            
            # 迭代step次时，实际已经完整的预测了testdata
            # 如sample/bs = 62.5需要迭代63次，step=62时已经迭代63次
            if step + 1 >= test_steps_fold:
                break  
        
        sess.close()
    
    test_preds = np.argmax(test_logits, axis=1)
    df_result["label"] = test_preds
    
    return df_result
    
    
    
# tf.reset_default_graph()    
# test_comments, test_comment_ids = read_file(config.test_dir, train=False)
# test_ids = convert_to_inputids(test_comments, word2id, 50)
# # test_loader = test_batch_iter(test_ids, rnn_config.test_batch_size)  
# model = BiRNN(config)
# # model_path = './best_models/rnnfold-1'
# test_result = test_model(config, model, test_ids)   